In [108]:
import pandas as pd
import numpy as np

In [120]:
#import data

trains=pd.read_csv('train.csv')
tests=pd.read_csv('test.csv')
details=pd.read_excel('details.xlsx')
subm=pd.read_csv('Sample_Solution.csv')

In [121]:
details

,ID,Unique ID (can not be used for predictions)
0,Gender,Sex of the applicant
1,DOB,Date of Birth of the applicant
2,Lead_Creation_Date,Date on which Lead was created
3,City_Code,Anonymised Code for the City
4,City_Category,Anonymised City Feature
5,Employer_Code,Anonymised Code for the Employer
6,Employer_Category1,Anonymised Employer Feature
7,Employer_Category2,Anonymised Employer Feature
8,Monthly_Income,Monthly Income in Dollars
9,Customer_Existing_Primary_Bank_Code,Anonymised Customer Bank Code


In [122]:
train=trains.drop(['DOB','City_Code','Employer_Code','ID','Source','Customer_Existing_Primary_Bank_Code','Employer_Category2','Lead_Creation_Date'], axis=1)
test=tests.drop(['DOB','City_Code','Employer_Code','ID','Source','Customer_Existing_Primary_Bank_Code','Employer_Category2','Lead_Creation_Date'], axis=1)

In [123]:
all_data=train.append(test)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_data['Gender'] = le.fit_transform(all_data['Gender'].astype('str'))

le2 = LabelEncoder()
all_data['Contacted'] = le2.fit_transform(all_data['Contacted'].astype('str'))

le3 = LabelEncoder()
all_data['City_Category'] = le3.fit_transform(all_data['City_Category'].astype('str'))

le4 = LabelEncoder()
all_data['Source_Category'] = le4.fit_transform(all_data['Source_Category'].astype('str'))

le5 = LabelEncoder()
all_data['Primary_Bank_Type'] = le5.fit_transform(all_data['Primary_Bank_Type'].astype('str'))

le6 = LabelEncoder()
all_data['Employer_Category1'] = le6.fit_transform(all_data['Employer_Category1'].astype('str'))

train = all_data[0:train.shape[0]]
test = all_data[train.shape[0]::]


In [124]:
train['EMI'].fillna(0, inplace=True)
train['Interest_Rate'].fillna(0, inplace=True)

train['Loan_Amount'].fillna(0, inplace=True)
train['Loan_Period'].fillna(0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [125]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

all_X = train.append(test)
all_X_imputed = DataFrameImputer().fit_transform(all_X)

train = all_X_imputed[0:train.shape[0]]
test = all_X_imputed[train.shape[0]::]

In [126]:
import lightgbm as lgb
light=lgb.LGBMClassifier()
light.fit(train.drop('Approved', axis=1),train['Approved'])
lpred=light.predict(test.drop('Approved', axis=1))

submissionfile=pd.DataFrame(columns=['ID','Approved'])
submissionfile['ID']=tests['ID']
submissionfile['Approved']=lpred
submissionfile.to_csv('mck.csv', index=False)